# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
# 打開瀏覽器

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from datetime import datetime

browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()




In [2]:
# 取得資料，丟到 BeautifulSoup 解析

html_source = browser.page_source

soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

target_obs = 'SO2'
end_date = datetime(2018,8,1)
time_format = '%Y/%m'
found = False
d = {}
for tr in table.find_all('tr')[1:]:
    
    for i, td in enumerate(tr.find_all('td')):
        
        if len(tr.find_all('td')) == 5:
            if i == 0:
                obs = td.text
                if obs == target_obs:
                    d.setdefault(obs, {})
                    found = True
                else:
                    found = False
            if i == 2:
                date = td.text
            if i == 3:
                value = td.text
                if found and (datetime.strptime(date, time_format) <= end_date):
                    d[obs][date] = value
                    
        
        if len(tr.find_all('td')) == 3:
            if i == 0:
                date = td.text
            if i == 1:
                value = td.text
                if found and (datetime.strptime(date, time_format) <= end_date):
                    d[obs][date] = value
print(d)

{'SO2': {'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40'}}


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [3]:
# 取得資料，丟到 BeautifulSoup 解析

html_source = browser.page_source

soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

target_obs = ['SO2', 'CO']
end_date = datetime(2018,8,1)
time_format = '%Y/%m'
found = False
d = {}
for tr in table.find_all('tr')[1:]:
    
    for i, td in enumerate(tr.find_all('td')):
        
        if len(tr.find_all('td')) == 5:
            if i == 0:
                obs = td.text
                if obs in target_obs:
                    d.setdefault(obs, {})
                    found = True
                else:
                    found = False
            if i == 2:
                date = td.text
            if i == 3:
                value = td.text
                if found and (datetime.strptime(date, time_format) <= end_date):
                    d[obs][date] = value
                    
        
        if len(tr.find_all('td')) == 3:
            if i == 0:
                date = td.text
            if i == 1:
                value = td.text
                if found and (datetime.strptime(date, time_format) <= end_date):
                    d[obs][date] = value
print(d)

{'SO2': {'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40'}, 'CO': {'2018/01': '0.34', '2018/02': '0.44', '2018/03': '0.40', '2018/04': '0.38', '2018/05': '0.34', '2018/06': '0.29', '2018/07': '0.21', '2018/08': '0.30'}}


In [4]:
browser.quit()